# ENR Région

Production de ENR par les régions. 

**Objectif **: appliquer les pourcentages de chaque type d'énergie aux consommations de la commune

** Source ** : https://www.data.gouv.fr/fr/datasets/production-regionale-annuelle-des-energies-renouvelables-2015-2017/

Ce jeu de données présente les productions régionales annuelles d'énergie renouvelable en France (GWh).

Pour le biométhane, il s'agit des quantités injectées dans les réseaux de gaz (réseaux de distribution et de transport) en France.

Selon la directive Européenne 2009/28/CE, la partie renouvelable est calculée comme suit :

* Hydraulique : On soustrait 70 % de la consommation liée au pompage.
* Bioénergies : La production des usines d'incinération d'ordures ménagères est comptabilisée à 50%. Les autres combustibles sont comptabilisés à 100 %.
* Eolien et solaire : La production est comptabilisée à 100 %.

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('data/odre/production-annuelle-regionale-enr (1).csv', 
                 sep=';', encoding='utf-8')
df = df.rename(columns={'Année': 'annee', 'Code INSEE région': 'insee', 
                        'Région': 'region',
                        'Production hydraulique renouvelable (GWh)': 'hydraulique',
       'Production bioénergies renouvelable (GWh)': 'bioenergies',
       'Production éolienne renouvelable (GWh)': 'eolienne',
       'Production solaire renouvelable (GWh)': 'solaire',
       'Production gaz renouvelable (GWh)': 'gaz',
       'Production totale renouvelable (GWh)': 'total', 
                        'Géo-shape région':'gsr',
       'Géo-point région':'gpr'})
df.head(3)

,annee,insee,region,hydraulique,bioenergies,eolienne,solaire,gaz,total,gsr,gpr
0,2015,32,Hauts-de-France,13.2,713.9,4966.2,126.0,15.4,5834.7,"{""type"": ""Polygon"", ""coordinates"": [[[4.141925...","49.9699454628, 2.77194454967"
1,2015,11,Ile-de-France,55.2,709.0,52.7,71.9,37.1,925.9,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"
2,2016,11,Ile-de-France,58.0,813.9,61.0,69.7,43.2,1045.8,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"


In [14]:
# à la fin en front on va utiliser que les données les plus récentes
# df2 = df[df['annee'] == 2017]
df2 = df
df2.head()

,annee,insee,region,hydraulique,bioenergies,eolienne,solaire,gaz,total,gsr,gpr
0,2015,32,Hauts-de-France,13.2,713.9,4966.2,126.0,15.4,5834.7,"{""type"": ""Polygon"", ""coordinates"": [[[4.141925...","49.9699454628, 2.77194454967"
1,2015,11,Ile-de-France,55.2,709.0,52.7,71.9,37.1,925.9,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"
2,2016,11,Ile-de-France,58.0,813.9,61.0,69.7,43.2,1045.8,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"
3,2017,93,Provence-Alpes-Côte d'Azur,7912.0,782.4,117.6,1528.3,0.0,10340.3,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[4....","43.9575890923, 6.06569381091"
4,2015,24,Centre-Val de Loire,99.2,322.0,1927.6,234.3,0.0,2583.1,"{""type"": ""Polygon"", ""coordinates"": [[[2.168887...","47.484799284, 1.68510929648"


In [8]:
df2.columns

Index(['annee', 'insee', 'region', 'hydraulique', 'bioenergies', 'eolienne',
       'solaire', 'gaz', 'total', 'gsr', 'gpr'],
      dtype='object')

In [15]:
# convertir en str 
df3 = df2[['annee', 'insee', 'region', 'hydraulique', 'bioenergies', 'eolienne',
       'solaire', 'gaz', 'total', 'gsr', 'gpr']].astype(str)
df3.head()

,annee,insee,region,hydraulique,bioenergies,eolienne,solaire,gaz,total,gsr,gpr
0,2015,32,Hauts-de-France,13.2,713.9,4966.2,126.0,15.4,5834.7,"{""type"": ""Polygon"", ""coordinates"": [[[4.141925...","49.9699454628, 2.77194454967"
1,2015,11,Ile-de-France,55.2,709.0,52.7,71.9,37.1,925.9,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"
2,2016,11,Ile-de-France,58.0,813.9,61.0,69.7,43.2,1045.8,"{""type"": ""Polygon"", ""coordinates"": [[[1.609615...","48.7093394536, 2.50408172628"
3,2017,93,Provence-Alpes-Côte d'Azur,7912.0,782.4,117.6,1528.3,0.0,10340.3,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[4....","43.9575890923, 6.06569381091"
4,2015,24,Centre-Val de Loire,99.2,322.0,1927.6,234.3,0.0,2583.1,"{""type"": ""Polygon"", ""coordinates"": [[[2.168887...","47.484799284, 1.68510929648"


# To SQL

In [11]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='root',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

In [16]:
# fillna(' ')
# read_csv dtype str
# limiter la taille de la chaine envoyée[:n] 

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO region_enr "
           " ( annee, insee, region, hydraulique, bioenergies, "
           " eolienne, solaire, gaz, total)"
           "VALUES (%s, %s, %s, %s, %s,    %s, %s, %s, %s)"
          )

    for i, r in df3.iterrows(): 
        
        cursor.execute(sql, 
                       (
                           r['annee'], r['insee'], r['region'], 
                           r['hydraulique'], r['bioenergies'], r['eolienne'], 
                           r['solaire'], r['gaz'], r['total']
                       )
                      )

connection.commit() 

# Consommation totale 

Afin de calculer le pourcentage de EnR sur le total de consommation